In [1]:
import datetime as dt
from IPython.display import display, JSON
from harmony import BBox, Client, Collection, Request, Dimension
from harmony.config import Environment
import os
import netCDF4 as nc4

In [2]:
# Tropics Bounding box
name = 'tropics'
minLon = -160
maxLon = -75
minLat = -10
maxLat = 10

In [3]:
harmony_client = Client(env=Environment.LOCAL)
myenv = 'local'

In [4]:
# Construct Harmony requests for UAT
def get_request(minLon, minLat, maxLon, maxLat):
    collection = Collection(id='C1215726323-GES_DISC') # M2I3NPASM in UAT 
    request = Request(
        collection = collection,
        spatial    = BBox(minLon, minLat, maxLon, maxLat),
        dimensions = [Dimension('lev', 800, 900)],
        max_results = 1
    )
    return request

In [5]:
# Submit Harmony requests, get back the file name of the downloaded result

def submit_request(request, name):
    # submit the job, save the jobID, print a URL to check job status in browser
    job_id = harmony_client.submit(request)
    print('http://localhost:3000/jobs/'+job_id)

    # show the progress bar for the job, get the result URL
    harmony_client.wait_for_processing(job_id, show_progress=True)
    urls = harmony_client.result_urls(job_id, show_progress=True)

    # create a local subdir, download the results 
    dir_name = './harmony_'+name
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    results = harmony_client.download_all(job_id, directory=dir_name, overwrite=True)

    # stash the file names for easy reference
    file_names = [f.result() for f in results]

    # return the first (only?) file name 
    return file_names[0]

In [6]:
# Get this subset from Harmony UAT
request = get_request(minLon, minLat, maxLon, maxLat)

In [13]:
if request.is_valid():
    harmony_fname = submit_request(request, name)
else:
    harmony_fname = 'Invalid_Request'

http://localhost:3000/jobs/c2335172-3caa-46d9-bf70-4ed9ed47714d


 [ Processing: 100% ] |###################################################| [|]


In [8]:
print(harmony_fname)

./harmony_tropics/M2I3NPASM.5.12.4_MERRA2_300.inst3_3d_asm_Np.20010101_subsetted.nc4


In [14]:
f=nc4.Dataset(harmony_fname)

In [16]:
f.getncattr('history_json')

'[{"$schema":"https:\\/\\/harmony.earthdata.nasa.gov\\/schemas\\/history\\/0.1.0\\/history-0.1.0.json","date_time":"2022-06-06T17:22:07.084+0000","program":"hyrax","version":"1.16.5-229","parameters":[{"request_url":"https:\\/\\/opendap.uat.earthdata.nasa.gov\\/collections\\/C1215726323-GES_DISC\\/granules\\/M2I3NPASM.5.12.4%3AMERRA2_300.inst3_3d_asm_Np.20010101.nc4.dap.nc4?dap4.ce=%2FQI%5B%5D%5B4%3A8%5D%5B160%3A200%5D%5B32%3A168%5D%3B%2FQV%5B%5D%5B4%3A8%5D%5B160%3A200%5D%5B32%3A168%5D%3B%2FSLP%5B%5D%5B160%3A200%5D%5B32%3A168%5D%3B%2Flev%5B4%3A8%5D%3B%2FQL%5B%5D%5B4%3A8%5D%5B160%3A200%5D%5B32%3A168%5D%3B%2FT%5B%5D%5B4%3A8%5D%5B160%3A200%5D%5B32%3A168%5D%3B%2FU%5B%5D%5B4%3A8%5D%5B160%3A200%5D%5B32%3A168%5D%3B%2FEPV%5B%5D%5B4%3A8%5D%5B160%3A200%5D%5B32%3A168%5D%3B%2FO3%5B%5D%5B4%3A8%5D%5B160%3A200%5D%5B32%3A168%5D%3B%2Ftime%3B%2FOMEGA%5B%5D%5B4%3A8%5D%5B160%3A200%5D%5B32%3A168%5D%3B%2Flat%5B160%3A200%5D%3B%2FRH%5B%5D%5B4%3A8%5D%5B160%3A200%5D%5B32%3A168%5D%3B%2FV%5B%5D%5B4%3A8%5D%5B160%3

In [18]:
!ncdump -v lev ./harmony_tropics/M2I3NPASM.5.12.4_MERRA2_300.inst3_3d_asm_Np.20010101_subsetted.nc4 | tail -4

data:

 lev = 900, 875, 850, 825, 800 ;
}


In [23]:
!ncdump -v lev ../data_samples/M2I3NPASM/MERRA2_300.inst3_3d_asm_Np.20010101.nc4 | tail -6


 lev = 1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 725, 700, 650, 
    600, 550, 500, 450, 400, 350, 300, 250, 200, 150, 100, 70, 50, 40, 30, 
    20, 10, 7, 5, 4, 3, 2, 1, 0.699999988079071, 0.5, 0.400000005960464, 
    0.300000011920929, 0.100000001490116 ;
}
